<a href="https://colab.research.google.com/github/MarioNavarrete/bigdata-colab/blob/main/bigdata_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BigData Tools in Colab!

The idea of this tutorial is provide you the classical BigData tools without using any of your own local resources.

In [32]:
!rm -r /content/bigdata-colab/
!git clone https://github.com/MarioNavarrete/bigdata-colab.git

Cloning into 'bigdata-colab'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 46 (delta 22), reused 38 (delta 14), pack-reused 0
Unpacking objects: 100% (46/46), done.


Preparing the dataset for this tutorial.


In [44]:
import pandas as pd
parquet_file = '/content/bigdata-colab/dataset/data.parquet'
data = pd.read_parquet(parquet_file)
data = data.dropna()
data.reset_index(inplace = True, drop = True)
data.to_csv('data.csv',index = False, header = False)
data.head()

,datetime,a,b,c,d
0,2016-09-30 00:00:01,0.1,1.21,20.06,13.35
1,2016-09-30 00:15:01,0.1,1.21,19.77,13.60
2,2016-09-30 00:30:01,0.1,1.21,19.57,13.60
3,2016-09-30 00:45:01,0.1,1.21,19.38,13.34
4,2016-09-30 01:00:01,0.1,1.21,19.13,13.60


Instal the enviroment

In [4]:
exec(open('/content/bigdata-colab/executable.py').read())

Active services:
2082 DataNode
2306 Jps
2195 NodeManager
1812 NameNode
1722 ResourceManager
2251 JobHistoryServer



## HDFS and MapReduce

In [6]:
!hadoop fs -ls

Move the local file into our local HDFS

In [15]:
!hadoop fs -put /content/data.csv data.csv
!hadoop fs -ls

put: `data.csv': File exists
Found 1 items
-rw-r--r--   1 root supergroup   39640162 2020-11-02 02:35 data.csv


A single file as we expected.

We already have created a very basic MapReduce job, and a bash script that run the needed commands to make it work. This should produce the min,max and the average of the second column, grouped by date.

In [22]:
!bash /content/bigdata-colab/mapreduce/mapreduce.sh

Deleted /user/root/Output_20201102
20/11/02 03:14:08 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/content/bigdata-colab/mapreduce/mapper.py, /content/bigdata-colab/mapreduce/reducer.py, /tmp/hadoop-unjar6375455253880408071/] [] /tmp/streamjob7106388717830391058.jar tmpDir=null
20/11/02 03:14:09 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/11/02 03:14:09 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/11/02 03:14:09 INFO mapred.FileInputFormat: Total input paths to process : 1
20/11/02 03:14:09 INFO mapreduce.JobSubmitter: number of splits:2
20/11/02 03:14:10 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1604284386574_0002
20/11/02 03:14:10 INFO impl.YarnClientImpl: Submitted application application_1604284386574_0002
20/11/02 03:14:10 INFO mapreduce.Job: The url to track the job: http://454a6f742a7d:8088/proxy/application_1604284386574_0002/
20/11/02 03:14:

Check the outputs in HDFS and copy to local

In [23]:
!hadoop fs -ls Out*

Found 2 items
-rw-r--r--   1 root supergroup          0 2020-11-02 03:14 Output_20201102/_SUCCESS
-rw-r--r--   1 root supergroup      19345 2020-11-02 03:14 Output_20201102/part-00000


In [24]:
!mkdir /content/mapreduce
!hadoop fs -get /user/root/Output_20201102 /content/mapreduce/output

20/11/02 03:16:01 WARN hdfs.DFSClient: DFSInputStream has been closed already
20/11/02 03:16:02 WARN hdfs.DFSClient: DFSInputStream has been closed already


In [25]:
columns = ['date','max_value','min_value','avg_value']

output_mapred = pd.read_csv('/content/mapreduce/output/part-00000', sep = '\t', names=columns)
output_mapred.date = pd.to_datetime(output_mapred.date)
output_mapred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546 entries, 0 to 545
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       546 non-null    datetime64[ns]
 1   max_value  546 non-null    float64       
 2   min_value  546 non-null    float64       
 3   avg_value  546 non-null    float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 17.2 KB


## Hive

Hive will be used simulating a terminal in this notebook.

I have some handy HiveQL statements that we will need to run one by one in the terminal.

#### Loading file from local path and running interactively.

In [28]:
allow_groupby = 'set hive.groupby.orderby.position.alias=true;'

create_statement = """
create table test (datetime timestamp, a double, b double, c double, d double) row format delimited fields terminated by ',';
"""

load_data = "LOAD DATA LOCAL INPATH '/content/data.csv' OVERWRITE INTO TABLE test;"

sql = """
select to_date(datetime),
avg(a) avg_a,
avg(b) avg_b,
avg(c) avg_c,
avg(d) avg_d,
min(a) min_a,
min(b) min_b,
min(c) min_c,
min(d) min_d,
max(a) max_a,
max(b) max_b,
max(c) max_c,
max(d) max_d
from test
group by 1;
"""

exit = "exit;"

In [26]:
!hive

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.7-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.7.0/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]

Logging initialized using configuration in jar:file:/content/apache-hive-2.3.7-bin/lib/hive-common-2.3.7.jar!/hive-log4j2.properties Async: true
Hive-on-MR is deprecated in Hive 2 and may not be available in the future versions. Consider using a different execution engine (i.e. spark, tez) or using Hive 1.X releases.
hive> set hive.groupby.orderby.position.alias=true;
hive> create table test (datetime timestamp, a double, b double, c double, d double) row format delimited fields terminated by ',';

#### Running the same steps using a .sql file

In [30]:
!hive -f /content/bigdata-colab/hive/script.sql

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.7-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.7.0/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]

Logging initialized using configuration in jar:file:/content/apache-hive-2.3.7-bin/lib/hive-common-2.3.7.jar!/hive-log4j2.properties Async: true
OK
Time taken: 7.643 seconds
Loading data to table default.test2
OK
Time taken: 1.643 seconds
OK
col_name	data_type	comment
datetime            	timestamp           	                    
a                   	double              	                    
b                   	double              	                    
c                   	double              	  

#### MySQL + Sqoop to Hive

First we need to load our data into our MySQL data base, and then grab all that data and move it into hive

In [34]:
!mysql -u root --password=password testdb < /content/bigdata-colab/mysql/load_data.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [37]:
!mysql -u root --password=password testdb -e "select * from test3 limit 5"

mysql: [Warning] Using a password on the command line interface can be insecure.
+---------------------+------+------+-------+-------+
| datetime            | a    | b    | c     | d     |
+---------------------+------+------+-------+-------+
| 2016-09-30 00:00:01 |  0.1 | 1.21 | 20.06 | 13.35 |
| 2016-09-30 00:15:01 |  0.1 | 1.21 | 19.77 |  13.6 |
| 2016-09-30 00:30:01 |  0.1 | 1.21 | 19.57 |  13.6 |
| 2016-09-30 00:45:01 |  0.1 | 1.21 | 19.38 | 13.34 |
| 2016-09-30 01:00:01 |  0.1 | 1.21 | 19.13 |  13.6 |
+---------------------+------+------+-------+-------+


In [39]:
!sqoop import --connect jdbc:mysql://localhost/testdb --username root --password password --table test3 --hive-import -m 1

20/11/02 03:42:23 INFO sqoop.Sqoop: Running Sqoop version: 1.4.7
20/11/02 03:42:23 WARN tool.BaseSqoopTool: Setting your password on the command-line is insecure. Consider using -P instead.
20/11/02 03:42:23 INFO tool.BaseSqoopTool: Using Hive-specific delimiters for output. You can override
20/11/02 03:42:23 INFO tool.BaseSqoopTool: delimiters with --fields-terminated-by, etc.
20/11/02 03:42:24 INFO manager.MySQLManager: Preparing to use a MySQL streaming resultset.
20/11/02 03:42:24 INFO tool.CodeGenTool: Beginning code generation
Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.
20/11/02 03:42:24 INFO manager.SqlManager: Executing SQL statement: SELECT t.* FROM `test3` AS t LIMIT 1
20/11/02 03:42:24 INFO manager.SqlManager: Executing SQL statement: SELECT t.* FROM `test3` AS t LIMIT 1
20/11/02 03:42:24 INFO o

In [41]:
sql3 = """set hive.groupby.orderby.position.alias=true;
select to_date(datetime),
avg(a) avg_a,
avg(b) avg_b,
avg(c) avg_c,
avg(d) avg_d,
min(a) min_a,
min(b) min_b,
min(c) min_c,
min(d) min_d,
max(a) max_a,
max(b) max_b,
max(c) max_c,
max(d) max_d
from test3
group by 1;
exit;
"""
!hive

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.7-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.7.0/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]

Logging initialized using configuration in jar:file:/content/apache-hive-2.3.7-bin/lib/hive-common-2.3.7.jar!/hive-log4j2.properties Async: true
Hive-on-MR is deprecated in Hive 2 and may not be available in the future versions. Consider using a different execution engine (i.e. spark, tez) or using Hive 1.X releases.
hive> select to_date(datetime), avg(a) avg_a, avg(b) avg_b, avg(c) avg_c, avg(d) avg_d, min(a) min_a, min(b) min_b, min(c) min_c, min(d) min_d, max(a) max_a, max(b) max_b, max(c) max_

## Spark

For this short tutorial, we are going to load the dataset directly from Hive, and also, directly from the parquet file.

In [50]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.enableHiveSupport().appName("bigdata-tools").getOrCreate()

#### Reading from Hive

In [51]:
spark.sql(sql[:-2]).show(10)

+--------------------------------+------------------+------------------+------------------+------------------+-----+-----+-----+-----+------+-----+-----+-----+
|to_date(default.test.`datetime`)|             avg_a|             avg_b|             avg_c|             avg_d|min_a|min_b|min_c|min_d| max_a|max_b|max_c|max_d|
+--------------------------------+------------------+------------------+------------------+------------------+-----+-----+-----+-----+------+-----+-----+-----+
|                      2017-08-11| 2.251536203522486|1.0629305283757302|11.317519569471639| 35.24196673189821|  0.1| -1.0| 6.63|  0.0|104.43| 3.23|28.15|49.78|
|                      2017-09-11| 9.066279166666662| 1.375249999999994|14.099666666666714|32.279079166666776|  0.1| -1.0| 4.15|  0.0|159.73|  5.4| 35.5|50.48|
|                      2017-01-06|14.131306089743381|1.6691826923077009|22.169535256410253|  34.2427644230769|  0.1|  0.6|10.31| 16.0|179.53| 3.69|44.37|58.87|
|                      2018-03-17|11.196

#### Reading parquet file

In [53]:
!hadoop fs -put /content/bigdata-colab/dataset/data.parquet data.parquet

spark_df = spark.read.parquet('/user/root/data.parquet')
spark_df.show(5)

put: `data.parquet': File exists
+-------------------+---+----+-----+-----+
|           datetime|  a|   b|    c|    d|
+-------------------+---+----+-----+-----+
|2016-09-30 00:00:01|0.1|1.21|20.06|13.35|
|2016-09-30 00:15:01|0.1|1.21|19.77| 13.6|
|2016-09-30 00:30:01|0.1|1.21|19.57| 13.6|
|2016-09-30 00:45:01|0.1|1.21|19.38|13.34|
|2016-09-30 01:00:01|0.1|1.21|19.13| 13.6|
+-------------------+---+----+-----+-----+
only showing top 5 rows



In [54]:
spark_df.createOrReplaceTempView("test4")
sql = """
select to_date(datetime),
avg(a) avg_a,
avg(b) avg_b,
avg(c) avg_c,
avg(d) avg_d,
min(a) min_a,
min(b) min_b,
min(c) min_c,
min(d) min_d,
max(a) max_a,
max(b) max_b,
max(c) max_c,
max(d) max_d
from test4
group by 1;
"""

In [58]:
spark.sql(sql[:-2]).show(5)

+-------------------------+------------------+------------------+------------------+------------------+-----+-----+-----+-----+------+-----+-----+-----+
|to_date(test4.`datetime`)|             avg_a|             avg_b|             avg_c|             avg_d|min_a|min_b|min_c|min_d| max_a|max_b|max_c|max_d|
+-------------------------+------------------+------------------+------------------+------------------+-----+-----+-----+-----+------+-----+-----+-----+
|               2017-08-11| 2.151995337995317|1.0629305283757302|11.229053613053628| 35.16296503496496|  0.1| -1.0| 6.63|  0.0|104.43| 3.23|28.15|49.78|
|               2017-09-11| 9.066279166666662| 1.375249999999994|14.099666666666714|32.279079166666776|  0.1| -1.0| 4.15|  0.0|159.73|  5.4| 35.5|50.48|
|               2017-01-06|14.131306089743381|1.6691826923077009|22.169535256410253|  34.2427644230769|  0.1|  0.6|10.31| 16.0|179.53| 3.69|44.37|58.87|
|               2018-03-17|11.196710982658898|1.8790693641618195|17.46684393063585